# Behavior Cloning Project

### Import Libraries

In [65]:
#keras
import keras
from keras.applications.resnet50 import ResNet50
from keras.layers import *
from keras.backend import clear_session
from keras.models import Model

# Helper libraries
import numpy as np
import pandas as pd
import sklearn
import math
import cv2
from PIL import Image

from sklearn.model_selection import train_test_split

import random
import csv
import matplotlib.pyplot as plt

from progressbar import *
import random

print('Keras Version: {}'.format(keras.__version__))

Keras Version: 2.2.4


### Import Data

In [71]:
def process_sample(smpl, correction_factor):
    camera_id = random.randint(0,2)
    
    if camera_id == 2:
        correction = -1*correction_factor
    else:
        correction = camera_id*correction_factor
        
    if random.random() >= 0.5:
        return [cv2.flip(cv2.imread('data/IMG/' + line[camera_id].split('/')[1]), 0), 
                (float(smpl[3])+correction)*(-1)]
    else:
        return [cv2.imread('data/IMG/' + line[camera_id].split('/')[1]), 
                float(smpl[3])+correction]

In [72]:
def generator(samples, batch_size=32, cf = 0.2):
    while True:
        random.shuffle(samples)
        for offset in range(0, len(samples), batch_size):
            subsmpls = samples[offset:offset+batch_size]
            images, measurements = zip(*[process_sample(smpl, cf) for smpl in subsmpls])
            X_train, y_train = (np.array(images), np.array(measurements))
            X_train, y_train = sklearn.utils.shuffle(X_train, y_train)
            yield (X_train, y_train)

In [43]:
reader = csv.reader(open('data/driving_log.csv'))
lines = [line for line in reader][1:]
train_lines, valid_lines = train_test_split(lines, test_size=0.2)

batch_size=48

In [40]:
## Process Images With Resnet and Save
x1 = Input(shape=(160,320,3))
x2 = Lambda(lambda x: (x/255)-0.5, input_shape=(160,320,3))(x1)
x3 = Cropping2D(cropping=((50,20), (0,0)))(x2)
resnet_model = ResNet50(include_top=False, weights='imagenet', input_tensor=x3, pooling=None)
resnet_model._make_predict_function()

for line in progressbar(lines):
    for camera_id in range(3):
        img = cv2.imread('data/IMG/' + line[camera_id].split('/')[1])
        prediction = resnet_model.predict(np.expand_dims(img, axis=0))[0]
        Image.fromarray(prediction, 'RGB').save('data/resnet/' + line[camera_id].split('/')[1])

100% (8036 of 8036) |####################| Elapsed Time: 1:29:56 Time:  1:29:56


### Define a Model

In [67]:
def create_model():
    model = keras.Sequential()
    
    model.add(Flatten(input_shape=(3, 10, 3)))

    model.add(Dense(48, activation='relu'))

    model.add(Dense(1))

    model.build()

    model.compile(optimizer=keras.optimizers.Adam(), loss='mse')
    
    return model

def create_transfer_learning_model(base_model):
    x = base_model.output
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x) 
    x = Dense(256, activation='relu')(x) 
    predictions = Dense(1)(x) 
    model = Model(input=base_model.input, output=predictions)

    model.compile(loss='mse', optimizer=keras.optimizers.Adam())

    return model

In [68]:
def train(train, valid, model):
    history = model.fit_generator(train, 
                steps_per_epoch=math.ceil(len(train_lines)/batch_size), 
                validation_data=valid, 
                validation_steps=math.ceil(len(valid_lines)/batch_size), 
                epochs=5, verbose=1, use_multiprocessing=True)
    return [model.evaluate_generator(valid, steps=len(valid_lines)), history]

### Save Model

In [69]:
def test_suite(model, correction_factor=0.2):
    if model is None:
        print("Specify a Model to Test")
        return
    elif not isinstance(model, list):
        if not isinstance(correction_factor, list):
            correction_factor = [correction_factor]
        mse, history = zip(*[train(generator(train_lines, batch_size = batch_size, cf = factor),
                              generator(valid_lines, batch_size = batch_size, cf = factor), model)
                             for factor in progressbar(correction_factor)])
    else:
        if not isinstance(correction_factor, list):
            correction_factor = [correction_factor]
        mse, history = zip(*[zip(*[train(generator(train_lines, batch_size = batch_size, cf = factor),
                              generator(valid_lines, batch_size = batch_size, cf = factor), m)
                        for factor in progressbar(correction_factor)]) for m in model])
    return mse, history

In [ ]:
clear_session()

## Process Images With Resnet and Save
x1 = Input(shape=(160,320,3))
x2 = Lambda(lambda x: (x/255)-0.5, input_shape=(160,320,3))(x1)
x3 = Cropping2D(cropping=((50,20), (0,0)))(x2)
base_model = ResNet50(include_top=False, weights='imagenet', input_tensor=x3, pooling=None)

model = create_transfer_learning_model(base_model)
model.summary()

train = generator(train_lines, batch_size = batch_size, cf = 0.15)
valid = generator(valid_lines, batch_size = batch_size, cf = 0.15)

history = model.fit_generator(train, 
                steps_per_epoch=math.ceil(len(train_lines)/batch_size), 
                validation_data=valid, 
                validation_steps=math.ceil(len(valid_lines)/batch_size), 
                epochs=5, verbose=1, use_multiprocessing=True)

# correction_factors = [0.1, 0.15, 0.2, 0.25, 0.30]

# mse, history = test_suite(model, correction_factor=correction_factors)

# plt.plot(correction_factors, mse[1])

/opt/anaconda3/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 160, 320, 3)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 160, 320, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
cropping2d_1 (Cropping2D)       (None, 90, 320, 3)   0           lambda_1[0][0]                   
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 96, 326, 3)   0           cropping2d_1[0][0]               
__________________________________________________________________________________________________
conv1 (Con

Epoch 1/5
  5/134 [>.............................] - ETA: 2:02:45 - loss: 2326.3508

Process ForkPoolWorker-25:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Process ForkPoolWorker-26:
  File "/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/opt/anaconda3/lib/python3.7/multiprocessing/queues.py", line 352, in get
    res = self._reader.recv_bytes()
  File "/opt/anaconda3/lib/python3.7/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/opt/anaconda3/lib/python3.7/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
  File "/opt/anaconda3/lib/python3.7/multiprocessing

  File "/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/opt/anaconda3/lib/python3.7/multiprocessing/queues.py", line 352, in get
    res = self._reader.recv_bytes()
  File "/opt/anaconda3/lib/python3.7/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/opt/anaconda3/lib/python3.7/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/opt/anaconda3/lib/python3.7/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt


### Output Model

In [63]:
model.save('my_model.h5')

ValueError: Fetch argument <tf.Variable 'dense_1/kernel:0' shape=(90, 48) dtype=float32_ref> cannot be interpreted as a Tensor. (Tensor Tensor("dense_1/kernel:0", shape=(90, 48), dtype=float32_ref) is not an element of this graph.)